In [1]:
import pandas as pd
import torch
from accelerate import Accelerator
from datasets import Dataset
import numpy as np

In [2]:
import os
os.environ["PYTORCH_MPS_HIGH_WATERMARK_RATIO"] = "0.0"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [3]:
# Optimizing by using GPU if available
device = "mps" if torch.backends.mps.is_available() else "cpu"
accelerator = Accelerator()
print(f"Using device: {device}")

Using device: mps


In [4]:
df = pd.read_csv("./data/b6_train_data.csv")
# turn into a Python list for tokenization
df["choices"] = df['choices'].apply(eval)
dataset = Dataset.from_pandas(df)

In [5]:
from transformers import AutoModelForMultipleChoice, AutoTokenizer
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMultipleChoice.from_pretrained(model_name).to(device)

# Wrap with `accelerate`
model = accelerator.prepare(model)
model.to(device)

Some weights of DistilBertForMultipleChoice were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForMultipleChoice(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
        

In [6]:
letter_to_number = {'A': 0, 'B': 1, 'C': 2, 'D': 3}


def get_number(ans):
    try:
        last_word = ans.split()[-1]  # Get the last word
        # Return mapped value or -1 if not found
        return letter_to_number.get(last_word, -1)
    except Exception as e:
        print(f"Error processing answer '{ans}': {e}")
        return -1  # Fallback value


def preprocess(examples):
    # Ensure choices are lists and pad to 4 choices
    examples["choices"] = [
        choice + [""] * (4 - len(choice)) if len(choice) < 4 else choice[:4]
        for choice in examples["choices"]
    ]

    # Number of choices per question (always 4 now)
    choice_lens = [4] * len(examples["choices"])

    # Expand questions to match the number of choices (4 per question)
    questions = [q for q_list in [[question] *
                                  4 for question in examples['question']] for q in q_list]
    choices = sum(examples["choices"], [])  # Flatten choices

    # Convert labels
    labels = [get_number(label) for label in examples['answer']]

    # Tokenize questions and choices as independent pairs
    tokenized_examples = tokenizer(
        list(zip(questions, choices)), truncation=True, padding="max_length", max_length=256
    )

    # Reshape data: Group every 4 choices together (for each question)
    reshaped_dict = {k: [] for k in tokenized_examples.keys()}
    start = 0
    for _ in range(len(examples["question"])):  # Iterate per question
        for k in tokenized_examples.keys():
            reshaped_dict[k].append(tokenized_examples[k][start: start + 4])
        start += 4

    # Ensure labels match the 4-choice structure
    reshaped_dict['labels'] = labels

    return reshaped_dict

In [7]:
idx = 0
tokenized_data = dataset.map(
    preprocess, batched=True, batch_size=8)

Map:   0%|          | 0/3963 [00:00<?, ? examples/s]

Error processing answer 'None': 'NoneType' object has no attribute 'split'
Error processing answer 'None': 'NoneType' object has no attribute 'split'
Error processing answer 'None': 'NoneType' object has no attribute 'split'
Error processing answer 'None': 'NoneType' object has no attribute 'split'
Error processing answer 'None': 'NoneType' object has no attribute 'split'
Error processing answer 'None': 'NoneType' object has no attribute 'split'
Error processing answer 'None': 'NoneType' object has no attribute 'split'
Error processing answer 'None': 'NoneType' object has no attribute 'split'
Error processing answer 'None': 'NoneType' object has no attribute 'split'
Error processing answer 'None': 'NoneType' object has no attribute 'split'
Error processing answer 'None': 'NoneType' object has no attribute 'split'
Error processing answer 'None': 'NoneType' object has no attribute 'split'
Error processing answer 'None': 'NoneType' object has no attribute 'split'
Error processing answer '

In [8]:
decoded_text = tokenizer.decode(
    tokenized_data[9]["input_ids"][0], skip_special_tokens=True)
print(decoded_text)

question : what value does the variable z have after all of the code above executes? int x ; int y ; int z ; x = 3 ; y = 4 ; z = + + x * y + + ; 9


In [9]:
from transformers import DataCollatorForMultipleChoice
from torch.utils.data import DataLoader


# Initialize collator
data_collator = DataCollatorForMultipleChoice(tokenizer=tokenizer)

In [10]:
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback
# Split your dataset into train and evaluation sets

train_test_split = tokenized_data.train_test_split(test_size=0.2, seed=42)
train_dataset = train_test_split["train"]
eval_dataset = train_test_split["test"]

In [11]:
train_dataset = train_dataset.filter(
    lambda example: example["labels"] != -1)
eval_dataset = eval_dataset.filter(
    lambda example: example["labels"] != -1)

Filter:   0%|          | 0/3170 [00:00<?, ? examples/s]

Filter:   0%|          | 0/793 [00:00<?, ? examples/s]

In [12]:
from transformers import Trainer
from evaluate import load

# Load a metric (F1-score in this case)
metric = load("accuracy")

# Define a custom compute_metrics function


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [13]:
# Freeze layers

for name, param in model.named_parameters():
    if "classifier" not in name:
        param.requires_grad = False

In [14]:
import shutil
import os

log_dir = "./logs"
if os.path.exists(log_dir):
    shutil.rmtree(log_dir)

In [15]:
from transformers import Trainer
import torch
from transformers import Trainer, TrainingArguments
torch.mps.empty_cache()


training_args = TrainingArguments(
    output_dir="./mcq_model",
    eval_strategy="epoch",
    max_grad_norm=1.0,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=2,
    save_strategy="epoch",
    logging_dir="./logs",
    learning_rate=1e-5,
    fp16=False,
    bf16=False,
    load_best_model_at_end=True,
    warmup_ratio=0.1,
    logging_steps=10,
    optim="adamw_torch",
    dataloader_pin_memory=False,       # Disable for MPS
    dataloader_num_workers=0,          # Must be 0 on MPS
    torch_compile=False
)


class DebugTrainer(Trainer):
    def training_step(self, model, inputs, nums_items_in_batch):
        outputs = model(**inputs)
        print("Loss:", outputs.loss)
        return outputs.loss


# Create the Trainer instance
trainer = DebugTrainer(
    model=model,                        # Pre-trained BERT model
    args=training_args,                 # Training arguments
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,        # Efficient batching
    compute_metrics=compute_metrics     # Custom metric
)

In [16]:
# Start the training
trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Loss: tensor(1.3802, device='mps:0', grad_fn=<NllLossBackward0>)


Epoch,Training Loss,Validation Loss,Accuracy
1,1.374700,1.385725,0.278834


Loss: tensor(1.3916, device='mps:0', grad_fn=<NllLossBackward0>)
Loss: tensor(1.3698, device='mps:0', grad_fn=<NllLossBackward0>)
Loss: tensor(1.3803, device='mps:0', grad_fn=<NllLossBackward0>)
Loss: tensor(1.3505, device='mps:0', grad_fn=<NllLossBackward0>)
Loss: tensor(1.4742, device='mps:0', grad_fn=<NllLossBackward0>)
Loss: tensor(1.3869, device='mps:0', grad_fn=<NllLossBackward0>)
Loss: tensor(1.4335, device='mps:0', grad_fn=<NllLossBackward0>)
Loss: tensor(1.4025, device='mps:0', grad_fn=<NllLossBackward0>)
Loss: tensor(1.3944, device='mps:0', grad_fn=<NllLossBackward0>)
Loss: tensor(1.3131, device='mps:0', grad_fn=<NllLossBackward0>)
Loss: tensor(1.3758, device='mps:0', grad_fn=<NllLossBackward0>)
Loss: tensor(1.2871, device='mps:0', grad_fn=<NllLossBackward0>)
Loss: tensor(1.4933, device='mps:0', grad_fn=<NllLossBackward0>)
Loss: tensor(1.3414, device='mps:0', grad_fn=<NllLossBackward0>)
Loss: tensor(1.3630, device='mps:0', grad_fn=<NllLossBackward0>)
Loss: tensor(1.4157, devi

In [ ]:
# Save the fine-tuned model
model.save_pretrained("./fine_tuned_model")

# Prediction


In [ ]:

test_df = pd.read_csv("./data/b6_test_data.csv")
test_df["choices"] = test_df['choices'].apply(eval)
fpttest_data = Dataset.from_pandas(test_df)

In [ ]:
def preprocess_predict(examples):
    # Ensure choices are lists and pad to 4 choices
    examples["choices"] = [
        choice + [""] * (4 - len(choice)) if len(choice) < 4 else choice[:4]
        for choice in examples["choices"]
    ]

    # Expand questions to match the number of choices (4 per question)
    questions = [q for q_list in [[question] *
                                  4 for question in examples['question']] for q in q_list]
    choices = sum(examples["choices"], [])  # Flatten choices

    # Tokenize questions and choices as independent pairs
    tokenized_examples = tokenizer(
        list(zip(questions, choices)), truncation=True, padding="max_length"
    )

    # Reshape data: Group every 4 choices together (for each question)
    reshaped_dict = {k: [] for k in tokenized_examples.keys()}
    start = 0
    for _ in range(len(examples["question"])):  # Iterate per question
        for k in tokenized_examples.keys():
            reshaped_dict[k].append(tokenized_examples[k][start: start + 4])
        start += 4

    return reshaped_dict

In [ ]:
tokenized_fpt = fpttest_data.map(
    preprocess_predict, batched=True, batch_size=8, load_from_cache_file=False)

print(tokenized_fpt[0])

Map:   0%|          | 0/1253 [00:00<?, ? examples/s]

{'task_id': 'k10171', 'question': 'Question: What will be output of the following c code?\n#include<stdio.h>\nint main()\n{\n    int a= sizeof(signed) +sizeof(unsigned);\n    int b=sizeof(const)+sizeof(volatile);\n    printf("%d",a+++b);\n    return 0;\n}', 'choices': ['10', '9', '8', 'Error'], 'input_ids': [[101, 3160, 1024, 2054, 2097, 2022, 6434, 1997, 1996, 2206, 1039, 3642, 1029, 1001, 2421, 1026, 2358, 20617, 1012, 1044, 1028, 20014, 2364, 1006, 1007, 1063, 20014, 1037, 1027, 2946, 11253, 1006, 2772, 1007, 1009, 2946, 11253, 1006, 27121, 1007, 1025, 20014, 1038, 1027, 2946, 11253, 1006, 9530, 3367, 1007, 1009, 2946, 11253, 1006, 20606, 1007, 1025, 6140, 2546, 1006, 1000, 1003, 1040, 1000, 1010, 1037, 1009, 1009, 1009, 1038, 1007, 1025, 2709, 1014, 1025, 1065, 102, 2184, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
import torch
from transformers import AutoModelForMultipleChoice, AutoTokenizer

# Set device to MPS or CUDA (if available)
device = torch.device("mps" if torch.backends.mps.is_available(
) else "cuda" if torch.cuda.is_available() else "cpu")

# Load the pretrained BERT model for multiple-choice tasks (no fine-tuning)
path = "./mcq_model/checkpoint-394/"
model = AutoModelForMultipleChoice.from_pretrained(path)
model.to(device)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(path)

# Define number to letter mapping
number_to_letter = {0: 'A', 1: 'B', 2: 'C', 3: 'D'}


def predict_single_token(tokenized_data):
    """
    Predicts the best answer choice for a given multiple-choice question.
    """
    input_ids = tokenized_data['input_ids']
    attention_mask = tokenized_data['attention_mask']

    # Ensure the input_ids and attention_mask have shape (batch_size, num_choices, sequence_length)
    input_ids = input_ids.to(device)  # Move to device (MPS or GPU)
    attention_mask = attention_mask.to(device)  # Same for attention_mask

    # Perform prediction without calculating gradients (in inference mode)
    model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        outputs = model(
            input_ids=input_ids, attention_mask=attention_mask)

    # Get the logits (predicted scores) for each choice
    logits = outputs.logits  # Shape: (batch_size, num_choices)
    print(logits)

    # Get the predicted index (which choice has the highest score)
    predictions = torch.argmax(logits, dim=1).tolist()

    # Map the predicted index to the corresponding letter (A, B, C, D)
    predicted_answers = [number_to_letter[prediction]
                         for prediction in predictions]

    return predicted_answers


# Example of a single tokenized input (representing a single multiple-choice question)
question = "What is the capital of United States?"
choices = ["Berlin", "Madrid", "Hanoi", "DC"]

# Tokenize the question and choices using the tokenizer, ensuring consistent padding
input_ids = []
attention_mask = []

for choice in choices:
    # Tokenize the question and choice pair
    encoded = tokenizer.encode_plus(
        question, choice,  # Question and each choice
        add_special_tokens=True,
        padding='max_length',  # Ensure consistent padding
        truncation=True,
        max_length=512,  # Adjust the max_length if necessary
        return_tensors="pt"
    )

    # Append the tokenized inputs
    input_ids.append(encoded['input_ids'])
    attention_mask.append(encoded['attention_mask'])

# Stack all the choices to create the batch
input_ids = torch.cat(input_ids, dim=0).to(
    device)  # Shape: (num_choices, sequence_length)
attention_mask = torch.cat(attention_mask, dim=0).to(
    device)  # Same shape as input_ids


# Prepare the inputs as a dictionary
tokenized_data = {
    'input_ids': input_ids.unsqueeze(0),  # Add batch dimension
    'attention_mask': attention_mask.unsqueeze(0),  # Add batch dimension
}

# Run the prediction on the single tokenized example
predictions = predict_single_token(tokenized_data)
print(predictions)  # Output: ['A', 'B', etc.]

tensor([[-0.0842, -0.0740, -0.0724, -0.0502]], device='mps:0')
['D']
